In [4]:
import torch
import numpy as np
from network import C3D_model
import cv2
torch.backends.cudnn.benchmark = True

def CenterCrop(frame, size):
    h, w = np.shape(frame)[0:2]
    th, tw = size
    x1 = int(round((w - tw) / 2.))
    y1 = int(round((h - th) / 2.))

    frame = frame[y1:y1 + th, x1:x1 + tw, :]
    return np.array(frame).astype(np.uint8)


def center_crop(frame):
    frame = frame[8:120, 30:142, :]
    return np.array(frame).astype(np.uint8)


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device being used:", device)

    with open('/Users/mosthusnejahan/Desktop/project_most/dataloaders/labels.txt', 'r') as f:
        class_names = f.readlines()
        f.close()
    # init model
    model = C3D_model.C3D(num_classes=4)
    checkpoint = torch.load('/Users/mosthusnejahan/Desktop/project_most/run/run_3/models/C3D-hmdb51_dataset_epoch-99.pth.tar', map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    model.train()

    # read video
    video = 'test/YouTube_smiles!_smile_h_nm_np1_fr_med_36.avi'
    #video = 'test/OldSchool_drink_h_nm_np1_fr_goo_4.avi'
    #video = 'test/Bill_Clinton_Statue_Unveiled_In_Kosovo_shake_hands_f_cm_np2_le_bad_0.avi'
    #video = 'test/Man_Who_Cheated_Himself_512kb_sit_u_cm_np1_fr_med_3.avi'
    cap = cv2.VideoCapture(video)
    retaining = True
    
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH);
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT); 
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('output/output2.mp4',fourcc, 15.0, (int(w),int(h)))
    
    
    #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #out = cv2.VideoWriter('output/outputVideo.avi',fourcc, 10.0, (160,120))
    
    #frame rate in the video file
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    fps=round(fps)
    print('\nframe rate=',fps)

    #Number of frames in the video file
    frames=[cap.get(cv2.CAP_PROP_FRAME_COUNT)]
    frames = [round(x) for x in frames]
    print('\nno of frames=',frames)
    
    
    clip = []
    while retaining:
        retaining, frame = cap.read()
        
        # Print the current frame
        #i=cap.get(cv2.CAP_PROP_POS_FRAMES)
        #i=round(i)
        #c=len(frame)
        #print('\ncurrent frame---',i)
        
    
        if not retaining and frame is None:
            continue
        tmp_ = center_crop(cv2.resize(frame, (171, 128)))
        tmp = tmp_ - np.array([[[90.0, 98.0, 102.0]]])
        clip.append(tmp)
        if len(clip) == 16:
            inputs = np.array(clip).astype(np.float32)
            inputs = np.expand_dims(inputs, axis=0)
            inputs = np.transpose(inputs, (0, 4, 1, 2, 3))
            inputs = torch.from_numpy(inputs)
            inputs = torch.autograd.Variable(inputs, requires_grad=False).to(device)
            with torch.no_grad():
                outputs = model.forward(inputs)

            probs = torch.nn.Softmax(dim=1)(outputs)
            label = torch.max(probs, 1)[1].detach().cpu().numpy()[0]
            print("Activity label {} ".format(class_names[label]))
            
            cv2.putText(frame, class_names[label].split(' ')[-1].strip(), (20, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 0, 255), 1)
            cv2.putText(frame, "prob: %.4f" % probs[0][label], (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (0, 0, 255), 1)
            clip.pop(0)

        #cv2.imshow('result', frame)
        out.write(frame)
        #print("output writting..")
        #cv2.waitKey(30)
    
    out.release()
    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()
    print("Test For Given Video Completed")



Device being used: cpu

frame rate= 30

no of frames= [78]
Activity label 1 drink
 
Activity label 3 sit
 
Activity label 2 shake_hands
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 2 shake_hands
 
Activity label 2 shake_hands
 
Activity label 3 sit
 
Activity label 3 sit
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 3 sit
 
Activity label 1 drink
 
Activity label 2 shake_hands
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 2 shake_hands
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 1 drink
 
Activity label 4 smile
 
Activity label 1 dr